# 🌍 Add New Language to DIGIT System

## 📋 Overview
This notebook helps you add a new language (Hindi, Tamil, Punjabi, etc.) to your DIGIT system in 4 easy steps:

1. **Authenticate** with the DIGIT Gateway
2. **Fetch** all English messages from the system
3. **Fill** translations in the generated Excel file
4. **Upload** translations back to the system

## 🎯 What You'll Need

- **Gateway Credentials**: Username, password, and access to the DIGIT system
- **Base URL**: Your system's gateway URL (e.g., `https://qa.digit.org`)
- **State Tenant ID**: State-level tenant (e.g., `pg`, `pb`, `mh`)
- **Target Tenant IDs**: City-level tenants where language will be enabled (e.g., `pg.citya`)
- **Language Name**: Display name for the language (e.g., `Hindi`, `Tamil`)
- **Locale Code**: Language code in format `language_COUNTRY` (e.g., `hi_IN`, `ta_IN`)

## 📊 Common Locale Codes

| Language | Locale Code | Native Name |
|----------|-------------|-------------|
| Hindi | `hi_IN` | हिंदी |
| Tamil | `ta_IN` | தமிழ் |
| Punjabi | `pa_IN` | ਪੰਜਾਬੀ |
| Bengali | `bn_IN` | বাংলা |
| Telugu | `te_IN` | తెలుగు |
| Marathi | `mr_IN` | मराठी |

---

## 🚀 Quick Start

**Step 1:** Authenticate with your credentials  
**Step 2:** Fetch localization data and download Excel template  
**Step 3:** Fill translations in the Excel file  
**Step 4:** Upload the translated file  
**Step 5:** Verify language appears in the application

---

**📖 For detailed instructions, see:** `HOW_TO_ADD_NEW_LANGUAGE.md`

In [32]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [33]:
# Import required libraries
import sys
import os
import json
import ipywidgets as widgets
from IPython.display import display, clear_output
import importlib

# Import functions from fetch_localization_preserve.py
import fetch_localization_preserve as flp

# Reload the module to get latest changes
importlib.reload(flp)
from unified_loader import UnifiedExcelReader, APIUploader, clean_nans

print("✓ All libraries imported successfully!")
print("✓ Functions loaded from fetch_localization_preserve.py")

✓ All libraries imported successfully!
✓ Functions loaded from fetch_localization_preserve.py


---

## 🔐 Step 1: Authenticate with Gateway

**IMPORTANT:** You must authenticate first before fetching or uploading data.

Fill in your credentials below and run the cell to authenticate.

### ⚠️ Required Permissions:
- User must have **MDMS_ADMIN** role or equivalent
- User must have access to localization API

In [34]:
# Authentication and Configuration
from unified_loader import APIUploader

# Initialize global variables
OUTPUT_FILE = "templates/localization.xlsx"
SHEET_NAME = "localization"
PARAMS = {}
LANGUAGE_NAME = ""
TARGET_TENANT_ID = []
UPLOADER = None  # Will be set after authentication

# Create authentication widgets
auth_base_url_input = widgets.Text(
    value='https://qa.digit.org',
    placeholder='e.g., https://qa.digit.org',
    description='Gateway URL:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

auth_username_input = widgets.Text(
    value='',
    placeholder='e.g., MDMSADMIN',
    description='Username:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

auth_password_input = widgets.Password(
    value='',
    placeholder='Enter your password',
    description='Password:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

auth_tenant_input = widgets.Text(
    value='pg',
    placeholder='e.g., pg, pb',
    description='State Tenant ID:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

auth_button = widgets.Button(
    description='🔐 Authenticate',
    button_style='success',
    tooltip='Click to authenticate',
    icon='lock',
    layout=widgets.Layout(width='200px', height='40px')
)

auth_output = widgets.Output()

def on_authenticate(b):
    """Authenticate with the gateway"""
    global UPLOADER, PARAMS
    
    with auth_output:
        clear_output()
        
        # Validate inputs
        if not auth_base_url_input.value.strip():
            print("❌ Gateway URL is required")
            return
        
        if not auth_username_input.value.strip():
            print("❌ Username is required")
            return
        
        if not auth_password_input.value.strip():
            print("❌ Password is required")
            return
        
        if not auth_tenant_input.value.strip():
            print("❌ State Tenant ID is required")
            return
        
        print("🔄 Authenticating with gateway...")
        print(f"   Gateway: {auth_base_url_input.value}")
        print(f"   Username: {auth_username_input.value}")
        print(f"   Tenant: {auth_tenant_input.value}")
        print()
        
        try:
            # Create APIUploader with authentication
            UPLOADER = APIUploader(
                base_url=auth_base_url_input.value.strip(),
                username=auth_username_input.value.strip(),
                password=auth_password_input.value.strip(),
                user_type='EMPLOYEE',
                tenant_id=auth_tenant_input.value.strip()
            )
            
            if UPLOADER and UPLOADER.authenticated:
                # Update PARAMS
                PARAMS['locale'] = "en_IN"
                PARAMS['tenantId'] = auth_tenant_input.value.strip()
                
                print("="*70)
                print("  ✅ AUTHENTICATION SUCCESSFUL!")
                print("="*70)
                print(f"\n👤 Logged in as: {UPLOADER.user_info.get('userName', 'Unknown')}")
                print(f"📧 Email: {UPLOADER.user_info.get('emailId', 'N/A')}")
                print(f"🏛️ Tenant: {UPLOADER.user_info.get('tenantId', 'N/A')}")
                print(f"\n🎭 Roles:")
                for role in UPLOADER.user_info.get('roles', []):
                    print(f"   • {role.get('name', 'Unknown')} ({role.get('code', 'N/A')})")
                print()
                print("="*70)
                print("  ➡️ Proceed to Step 2: Fetch Localization Data")
                print("="*70)
            else:
                print("❌ Authentication failed. Please check your credentials.")
        
        except Exception as e:
            print(f"❌ Error during authentication: {str(e)}")
            import traceback
            traceback.print_exc()

auth_button.on_click(on_authenticate)

# Display authentication UI
display(widgets.VBox([
    widgets.HTML(value="<h3>🔐 Gateway Authentication</h3>"),
    widgets.HTML(value="<p style='color: #666;'><i>Authenticate first to access all features</i></p>"),
    widgets.HTML("<br>"),
    auth_base_url_input,
    auth_username_input,
    auth_password_input,
    auth_tenant_input,
    widgets.HTML("<p style='font-size: 11px; color: #ff6600;'><i>⚠️ Credentials are used only for this session and not stored</i></p>"),
    widgets.HTML("<br>"),
    auth_button,
    auth_output
]))

print("✓ Authentication UI created!")

✓ Authentication UI created!


---

## 📥 Step 2: Fetch Localization Data

**Prerequisites:** You must be authenticated (Step 1) before proceeding.

Fill in the form below and click **"Fetch Data"** to:
1. Fetch all English messages from your system
2. Generate an Excel template with your new language column
3. Download the Excel file for translation

### 📝 Field Instructions:

- **Target Tenant IDs**: Cities where language will be enabled (comma-separated: `pg.citya, pg.cityb`)
- **New Language**: Display name users will see (e.g., `Hindi`, `Tamil`)
- **New Locale Code**: Format `language_COUNTRY` (e.g., `hi_IN`, `ta_IN`)

### ⚠️ Important:
- The system fetches **ALL English messages** (typically 10,000+ messages)
- You must translate **ALL messages** - empty translations will cause upload errors
- Keep the downloaded Excel file safe - you'll need it for upload

In [35]:
# Create UI widgets for fetching data
target_tenant_id_input = widgets.Text(
    value='',
    placeholder='e.g., pg.citya, pg.cityb, pg.cityc (comma-separated for multiple)',
    description='Target Tenant IDs:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

language_input = widgets.Text(
    value='',
    placeholder='e.g., Hindi, Tamil, Punjabi (NEW language you want to add)',
    description='New Language:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

locale_code_input = widgets.Text(
    value='',
    placeholder='e.g., hi_IN, ta_IN, pa_IN (code for NEW language)',
    description='New Locale Code:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

fetch_output_label = widgets.HTML(value="")

fetch_button = widgets.Button(
    description='📥 Fetch Data',
    button_style='primary',
    tooltip='Click to fetch localization data',
    icon='download',
    layout=widgets.Layout(width='200px', height='40px')
)

def process_localization_data(button):
    """Fetch localization data and generate Excel template"""
    global LANGUAGE_NAME, TARGET_TENANT_ID
    
    with fetch_output_label:
        clear_output(wait=True)
        
        # Check authentication first
        if not UPLOADER or not UPLOADER.authenticated:
            fetch_output_label.value = """
            <div style='background-color: #f8d7da; padding: 15px; border-radius: 5px; border: 1px solid #f5c6cb;'>
                <h3 style='color: #721c24;'>❌ Authentication Required</h3>
                <p>Please complete <strong>Step 1: Authenticate</strong> before fetching data.</p>
            </div>
            """
            return
        
        # Get input values
        locale_code = locale_code_input.value.strip()
        LANGUAGE_NAME = language_input.value.strip()
        
        # Parse comma-separated tenant IDs
        target_tenant_input = target_tenant_id_input.value.strip()
        TARGET_TENANT_ID = [t.strip() for t in target_tenant_input.split(',') if t.strip()]
        
        # Validate inputs
        if not TARGET_TENANT_ID:
            fetch_output_label.value = "<p style='color: red;'>❌ Error: Target Tenant ID is required</p>"
            return
        
        if not locale_code:
            fetch_output_label.value = "<p style='color: red;'>❌ Error: Locale code is required</p>"
            return
            
        if not LANGUAGE_NAME:
            fetch_output_label.value = "<p style='color: red;'>❌ Error: Language name is required</p>"
            return
        
        # Display progress
        target_tenants_display = ', '.join(TARGET_TENANT_ID)
        
        print(f"[INFO] Fetching localization data...")
        print(f"   Gateway: {UPLOADER.base_url}")
        print(f"   State Tenant: {PARAMS['tenantId']}")
        print(f"   Target Tenants: {target_tenants_display}")
        print(f"   Language: {LANGUAGE_NAME} ({locale_code})")
        print()
        
        try:
            # Construct API URL
            full_url = f"{UPLOADER.base_url}/localization/messages/v1/_search"
            
            # Step 1: Read existing translations
            print("[1/4] Reading existing translations...")
            existing_translations = flp.read_existing_translations(OUTPUT_FILE, SHEET_NAME)
            
            # Step 2: Fetch data from API
            print(f"[2/4] Fetching all English messages from API...")
            api_response = flp.fetch_localization_data(full_url, params=PARAMS)
            
            if not api_response:
                fetch_output_label.value = """<p style='color: red;'>❌ Failed to fetch data from API.</p>"""
                return
            
            # Step 3: Parse messages
            print("[3/4] Parsing messages...")
            messages = flp.parse_messages(api_response)
            
            if not messages:
                fetch_output_label.value = "<p style='color: red;'>❌ No messages found in response.</p>"
                return
            
            print(f"   Found {len(messages)} messages")
            
            # Step 4: Create DataFrame and merge translations
            print(f"[4/4] Creating Excel with locale '{locale_code}'...")
            df = flp.create_dataframe(messages, locale_code)
            df = flp.merge_translations(df, existing_translations)
            
            # Save to Excel
            print(f"\n   Saving to {OUTPUT_FILE}...")
            flp.preserve_validations_and_save(df, OUTPUT_FILE, SHEET_NAME)
            
            # Verify file
            if os.path.exists(OUTPUT_FILE):
                file_size = os.path.getsize(OUTPUT_FILE) / 1024
                print(f"   ✓ File saved successfully! Size: {file_size:.2f} KB")
                
                fetch_output_label.value = f"""
                <div style='background-color: #D4EDDA; padding: 15px; border-radius: 5px; border: 1px solid #C3E6CB;'>
                    <h3 style='color: #155724;'>✅ Excel Template Generated!</h3>
                    <p>📁 File: {OUTPUT_FILE}</p>
                    <p>📊 Total messages: {len(df):,}</p>
                    <p>🌐 Locale: {locale_code}</p>
                    <p>💾 File size: {file_size:.2f} KB</p>
                    <hr>
                    <p><strong>📝 Next Step:</strong></p>
                    <ol>
                        <li>Download the file using the link below</li>
                        <li>Open in Excel and fill the <strong>Message</strong> column with translations</li>
                        <li>Make sure NO cells are left empty</li>
                        <li>Come back to <strong>Step 3</strong> to upload</li>
                    </ol>
                </div>
                """
                
                # Display download link
                from IPython.display import FileLink
                print("\n" + "="*60)
                display(FileLink(OUTPUT_FILE, result_html_prefix="📥 Download file: "))
                print("="*60)
            
        except Exception as e:
            fetch_output_label.value = f"<p style='color: red;'>❌ Error: {str(e)}</p>"
            import traceback
            traceback.print_exc()

fetch_button.on_click(process_localization_data)

# Display fetch UI
display(widgets.VBox([
    widgets.HTML(value="<h3>📥 Fetch Localization Data</h3>"),
    widgets.HTML(value="<p style='color: #666;'><i>Enter details for the new language you want to add</i></p>"),
    widgets.HTML("<br>"),
    target_tenant_id_input,
    language_input,
    locale_code_input,
    widgets.HTML(value="""
        <div style='background-color: #e7f3ff; padding: 10px; margin: 10px 0; border-left: 4px solid #2196F3; border-radius: 4px;'>
            <strong>ℹ️ Locale Code Format:</strong>
            <ul style='margin: 5px 0; padding-left: 20px;'>
                <li><strong>language</strong> = lowercase 2-letter code (en, hi, ta, bn, etc.)</li>
                <li><strong>COUNTRY</strong> = uppercase 2-letter code (IN, US, GB, etc.)</li>
            </ul>
            <strong>Examples:</strong> en_IN (English-India), hi_IN (Hindi-India), ta_IN (Tamil-India)
        </div>
    """),
    widgets.HTML("<br>"),
    fetch_button,
    fetch_output_label
]))

print("✓ Fetch UI created!")

✓ Fetch UI created!


## Run the Application

Execute the cell below to display the interactive UI:

In [28]:
# Display the UI
display(widgets.VBox([
    widgets.HTML(value="<h2>Localization Data Fetcher</h2><p>Fill in the details below and click 'Fetch Data' to process:</p>"),
    base_url_input,
    tenant_id_input,
    target_tenant_id_input,
    language_input,
    locale_code_input,
    widgets.HTML(value="""
        <div style='background-color: #e7f3ff; padding: 10px; margin: 10px 0; border-left: 4px solid #2196F3; border-radius: 4px;'>
            <strong>ℹ️ Locale Code Format:</strong>
            <ul style='margin: 5px 0; padding-left: 20px;'>
                <li><strong>language</strong> = lowercase 2-letter code (en, hi, ta, bn, etc.)</li>
                <li><strong>COUNTRY</strong> = uppercase 2-letter code (IN, US, GB, etc.)</li>
            </ul>
            <strong>Examples:</strong> en_IN (English-India), hi_IN (Hindi-India), ta_IN (Tamil-India)
        </div>
    """),
    fetch_button,
    output_label
]))

---

## 📤 Step 3: Upload Translated File

**Prerequisites:** 
- You must be authenticated (Step 1)
- You must have downloaded and filled the Excel file (Step 2)

After filling translations in the Excel file:

1. Click **"Choose Files"** and select your filled Excel file
2. Click **"Validate & Save File"** button
3. The system will check for empty messages and save the file

### ✅ What Gets Validated:
- All message fields must be filled (no empty translations)
- Required columns (Module, Code, Locale, Message) must be present

### ⚠️ Common Errors:
- **"message must not be empty"** - Some translations are blank, fill all cells
- **"Invalid file format"** - Use the Excel file downloaded from Step 2

In [37]:
def validate_excel_file(file_content, filename, schema_file, display_name):
    """
    Validate Excel file content before saving
    Returns tuple: (is_valid, error_messages)
    """
    try:
        # Load ExcelValidator
        import sys
        sys.path.insert(0, os.path.abspath('.'))
        from mdms_validator import MDMSValidator as ExcelValidator
        
        # Create temp file for validation
        temp_dir = 'temp_validation'
        os.makedirs(temp_dir, exist_ok=True)
        temp_path = os.path.join(temp_dir, filename)
        
        # Write content to temp file
        with open(temp_path, 'wb') as f:
            f.write(file_content)
        
        # Validate
        validator = ExcelValidator(schemas_dir="schemas", templates_dir=temp_dir)
        result = validator.validate_file(filename, schema_file)
        
        # Clean up temp file
        os.remove(temp_path)
        
        if result['valid']:
            return True, []
        else:
            # Format error messages
            error_messages = []
            errors_by_sheet = {}
            
            for error in result['errors']:
                sheet = error.get('sheet', 'Unknown')
                if sheet not in errors_by_sheet:
                    errors_by_sheet[sheet] = []
                errors_by_sheet[sheet].append(error)
            
            for sheet_name, sheet_errors in errors_by_sheet.items():
                error_messages.append(f"\n   Sheet: {sheet_name}")
                for i, error in enumerate(sheet_errors[:3], 1):  # Show first 3 errors per sheet
                    if 'row' in error:
                        error_messages.append(
                            f"      {i}. Row {error['row']}, Column '{error['column']}'"
                        )
                        error_messages.append(f"         Error: {error['message']}")
                    else:
                        msg = error['message']
                        if msg.startswith(f"Sheet '{sheet_name}': "):
                            msg = msg.replace(f"Sheet '{sheet_name}': ", "")
                        error_messages.append(f"      {i}. {msg}")
                
                if len(sheet_errors) > 3:
                    error_messages.append(f"      ... and {len(sheet_errors) - 3} more errors")
            
            return False, error_messages
            
    except ImportError:
        # If validator not available, skip validation
        print(f"   ⚠️  Validator not available for {display_name}, saving without validation")
        return True, []
    except Exception as e:
        # If validation fails for any reason, show warning but allow save
        print(f"   ⚠️  Validation error for {display_name}: {str(e)}")
        return True, []

# Create file upload widget
localization_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

upload_validate_button = widgets.Button(
    description='⬆️ Validate & Upload File',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px')
)

upload_output = widgets.Output()

def on_validate_upload(b):
    """Handler for validating and uploading the localization file"""
    global UPLOADED_LOCALIZATION_FILE
    
    with upload_output:
        clear_output()
        
        if not localization_file_upload.value:
            print("❌ No file selected. Please select a file first.")
            return
        
        print("="*70)
        print("  📋 VALIDATING UPLOADED FILE")
        print("="*70)
        print()
        
        uploaded_file = localization_file_upload.value[0]
        content = uploaded_file['content']
        filename = uploaded_file['name']
        
        print(f"[Localization File]")
        print(f"   📄 File: {filename}")
        print(f"   🔍 Validating...", end=" ")
        
        # Validate the file
        # Note: You need to specify the correct schema file for localization
        schema_file = 'localization_schema.yaml'
        display_name = 'Localization'
        
        is_valid, errors = validate_excel_file(content, filename, schema_file, display_name)
        
        if is_valid:
            print("✅ PASSED")
            # Save file to upload directory
            os.makedirs('upload', exist_ok=True)
            upload_path = os.path.join('upload', filename)
            with open(upload_path, 'wb') as f:
                f.write(content)
            
            UPLOADED_LOCALIZATION_FILE = upload_path
            
            print(f"   💾 Saved to: {upload_path}")
            print()
            print("="*70)
            print("  📊 VALIDATION SUMMARY")
            print("="*70)
            print(f"\n   ✅ File validated successfully!")
            print(f"\n   ➡️  File ready to be uploaded to the system.")
            print(f"   💾 File saved to: {upload_path}")
            print("="*70)
        else:
            print("❌ FAILED")
            print("   ⚠️  File NOT saved due to validation errors:")
            for error in errors:
                print(error)
            print()
            print("="*70)
            print("  📊 VALIDATION SUMMARY")
            print("="*70)
            print(f"\n   ❌ Validation failed")
            print("\n   💡 Fix the errors in your Excel file and try uploading again.")
            print("="*70)

upload_validate_button.on_click(on_validate_upload)

# Initialize global variable
UPLOADED_LOCALIZATION_FILE = None

# Display the upload UI
display(widgets.VBox([
    widgets.HTML(value="<h3>📁 Upload Updated Localization File</h3>"),
    widgets.HTML(value="<p><i>After filling in translations, upload your file here. It will be validated before being accepted.</i></p>"),
    localization_file_upload,
    widgets.HTML("<br>"),
    upload_validate_button,
    upload_output
]))

print("✓ File upload and validation UI created!")

✓ File upload and validation UI created!


---

## 🚀 Step 4: Upload Translations to System

**Prerequisites:**
- Authentication successful (Step 1)
- File validated and saved (Step 3)

Run the cell below to:
1. **Upload** all translation messages to the localization API
2. **Update** tenant configuration to enable the new language in the UI

### 📊 What Happens:
- All messages are sent to `/localization/messages/v1/_upsert` API
- Each tenant in your Target Tenant IDs list gets the language added to their dropdown
- Users will see the new language option in the UI

### ✅ Expected Output:
```
[1/2] Uploading localization messages...
✅ Created: 12,752 messages

[2/2] Updating tenant languages for 3 tenant(s)...
✅ Language added to 3 tenant(s)!
```

### ❌ If Upload Fails:
- Check the error file: `errors/FAILED_RECORDS.xlsx`
- Most common issue: Empty message fields
- Fix errors in Excel and re-upload from Step 3

In [38]:
# Load Localization and Update Tenant Languages
print("="*60)
print("[STEP 4: UPLOAD TRANSLATIONS]")
print("="*60)

# Check authentication
if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Authentication required!")
    print("   Please complete Step 1: Authenticate with Gateway")
else:
    # Check if file was uploaded
    if not UPLOADED_LOCALIZATION_FILE:
        print("❌ No file uploaded!")
        print("   Please complete Step 3: Upload Translated File")
    else:
        try:
            # Initialize reader
            reader = UnifiedExcelReader(UPLOADED_LOCALIZATION_FILE)
            
            # Read localization data
            localization_data = reader.read_localization()
            
            if not localization_data:
                print("\n❌ No localization data found in the file")
                print("   Make sure the Excel has data in the 'localization' sheet")
            else:
                print(f"\n[INFO] Loaded {len(localization_data)} translations")
                
                # Step 1: Upload Localization using dedicated localization API
                print("\n[1/2] Uploading localization messages...")
                result = UPLOADER.create_localization_messages(
                    localization_list=clean_nans(localization_data),
                    tenant=PARAMS['tenantId']
                )
                
                if result['failed'] == 0:
                    print("\n✅ [SUCCESS] Localization messages uploaded successfully!")
                else:
                    print(f"\n⚠️ [WARNING] Some localizations failed. Check errors above.")
                
                # Step 2: Update tenant languages
                if TARGET_TENANT_ID and LANGUAGE_NAME:
                    print(f"\n[2/2] Updating tenant languages for {len(TARGET_TENANT_ID)} tenant(s)...")
                    
                    # Get locale code from the localization data
                    locale_code = localization_data[0].get('locale', '') if localization_data else ''
                    
                    if locale_code and LANGUAGE_NAME:
                        language_result = UPLOADER.update_tenant_language(
                            tenant_ids=TARGET_TENANT_ID,
                            language_label=LANGUAGE_NAME,
                            language_value=locale_code,
                            state_tenant=PARAMS['tenantId']
                        )
                        
                        if language_result['updated'] > 0:
                            print(f"\n✅ [SUCCESS] Language added to {language_result['updated']} tenant(s)!")
                            print(f"\n🎉 Process complete! Proceed to Step 5 to verify.")
                        elif language_result['skipped'] > 0:
                            print(f"\n[INFO] Language already exists in all tenants (skipped {language_result['skipped']})")
                        
                        if language_result['failed'] > 0:
                            print(f"\n⚠️ [WARNING] Failed to update {language_result['failed']} tenant(s)")
                    else:
                        print("\n⚠️ [WARNING] Could not update tenant languages - locale code or language name missing")
                else:
                    print("\n⚠️ [INFO] Skipping tenant language update - no target tenants or language name specified")
                    print("   Make sure you completed Step 2: Fetch Localization Data")
                    
        except Exception as e:
            print(f"\n❌ [ERROR] Upload process failed: {str(e)}")
            import traceback
            traceback.print_exc()

[STEP 4: UPLOAD TRANSLATIONS]

❌ No localization data found in the file
   Make sure the Excel has data in the 'localization' sheet


---

## ✅ Step 5: Verify Language in Application

After successful upload, verify the new language is working:

### 1️⃣ Login to Application
- Go to your DIGIT application URL (e.g., `https://qa.digit.org/employee`)
- Login with your credentials

### 2️⃣ Find Language Selector
- Look for the language dropdown (usually in header or profile menu)
- Click on it to see available languages

### 3️⃣ Select Your Language
- You should see your new language in the list (e.g., "Hindi", "Tamil")
- Click to switch to that language

### 4️⃣ Verify Translations
- Navigate through the application
- Check that UI labels, buttons, forms are translated
- Verify complaint types, department names appear in the new language

### 🔍 Troubleshooting:

**Language not in dropdown?**
- Check if tenant language update succeeded in Step 4
- Verify you're logged into the correct tenant (city)
- Try clearing browser cache and reloading

**Translations not appearing?**
- Some messages may be cached - wait a few minutes
- Clear browser cache
- Check if message codes match (Code column in Excel)

**Some messages still in English?**
- Those messages may not have been fetched in Step 2
- Some messages are added dynamically by modules
- You may need to add them manually via MDMS

---

## 🎉 Success!

If you can see and use the new language in your application, congratulations! You've successfully added a new language to your DIGIT system.

### 📚 Next Steps:
- Test all modules (PGR, Property Tax, etc.)
- Gather feedback from native speakers
- Update translations if needed (repeat Step 2-4)
- Add the language to other tenants/cities

---